In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15958965417122060732
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5726273536
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12502619072871558825
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6"
]


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder


def cutout(image, label, probability=0.5):
    if np.random.rand() < probability:
        h, w = image.shape[:2]
        size = np.random.randint(w // 2)
        x1 = np.random.randint(w)
        y1 = np.random.randint(h)
        x2 = np.clip(x1 + size, 0, w)
        y2 = np.clip(y1 + size, 0, h)
        image[y1:y2, x1:x2, :] = np.random.rand(y2 - y1, x2 - x1, 3)
    return image, label

df = pd.read_csv("train.csv")
label_encoder = LabelEncoder()
artist_df = df.copy()
artist_train = label_encoder.fit_transform(df['artist'].values)


artist_df['num'] = artist_train
artist_df = artist_df.drop('id', axis=1)
artist_df = artist_df.drop('img_path', axis=1)


artist_df.set_index('num', inplace=True)
artist_df = artist_df.sort_index()


artist_test_dic = artist_df['artist'].to_dict()


# 데이터 노이즈 수정
df.loc[df['id'] == 3896, 'artist'] = 'Titian'
df.loc[df['id'] == 3986, 'artist'] = 'Alfred Sisley'

X_train = df

#X_train, X_val, y_train, y_val = train_test_split(df, df['artist'].values, test_size=0.1)

print("Number of X_train: ", len(X_train))

Number of X_train:  5911


In [5]:
class_weights = class_weight.compute_class_weight('balanced', classes=pd.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(244,244,3), input_tensor=None, pooling=None)

x = GlobalAveragePooling2D()(base_model.output)
predictions = Dense(50, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

LearningRate = 1e-3

optimizer = Adam(learning_rate=LearningRate)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['acc'])

CP = ModelCheckpoint(filepath='model/' +
                     'ResNet50-Sigmoid-{epoch:03d}-{loss:.4f}-{val_loss:.4f}.hdf5',
                     monitor='val_loss', verbose=1, save_best_only=True, mode='min')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=0.00005)
CALLBACK = [CP, LR]

early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True,
)


DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    data_format="channels_last",
    validation_split=0.10
)


TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    dataframe=X_train, x_col='img_path', y_col='artist',
    target_size=(244, 244), 
    class_mode='categorical',
    batch_size=32, shuffle=True,
    subset="training",
    preprocessing_function=cutout
)

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    dataframe=X_train, x_col='img_path', y_col='artist',
    target_size=(244, 244), 
    class_mode='categorical',
    batch_size=32, shuffle=True,
    subset="validation",
    preprocessing_function=cutout
)

Found 5320 validated image filenames belonging to 50 classes.
Found 591 validated image filenames belonging to 50 classes.


In [6]:
history = model.fit(
    TRAIN_GENERATOR,
    epochs=200,
    callbacks=[early_stopping],
    shuffle=True,
    validation_data=VALID_GENERATOR,
    class_weight=class_weights_dict
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/200
167/167 [==============================] - 117s 663ms/step - loss: 5.2329 - acc: 0.1842 - val_loss: 18.5790 - val_acc: 0.0372
Epoch 2/200
167/167 [==============================] - 63s 375ms/step - loss: 4.1067 - acc: 0.2669 - val_loss: 8.1499 - val_acc: 0.0355
Epoch 3/200
167/167 [==============================] - 63s 378ms/step - loss: 3.6257 - acc: 0.3305 - val_loss: 5.2258 - val_acc: 0.0169
Epoch 4/200
167/167 [==============================] - 63s 375ms/step - loss: 3.2453 - acc: 0.3805 - val_loss: 5.6478 - val_acc: 0.0389
Epoch 5/200
167/167 [==============================] - 62s 372ms/step - loss: 2.9904 - acc: 0.4182 - val_loss: 4.8331 - val_acc: 0.0745
Epoch 6/200
167/167 [==============================] - 63s 374ms/step - loss: 2.7409 - acc: 0.4474 - val_loss: 2.6913 - val_acc: 0.3113
Epoch 7/200
167/167 [=======================

In [7]:
X_test = pd.read_csv("test.csv")

DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    data_format="channels_last"
)

TEST_GENERATOR = DATAGEN_TEST.flow_from_dataframe(
    dataframe=X_test,
    x_col='img_path',
    y_col=None,
    target_size=(244, 244),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False
)

TEST_Prediction = model.predict(TEST_GENERATOR, verbose=1)

Found 12670 validated image filenames.
396/396 [==============================] - 134s 337ms/step


In [8]:
artist_num = np.argmax(TEST_Prediction, axis=1)
num_list = pd.DataFrame(artist_num, columns=['artist'])

artist_name = []
for i in num_list['artist']:
    artist_name.append(artist_test_dic[i])

last_ans = pd.read_csv("sample_submission.csv")

last_ans.rename(columns={'artist':'artist_name'}, inplace=True)

last_ans['artist'] = artist_name
last_ans.drop('artist_name', axis=1, inplace=True)

last_ans.to_csv("ResNet50_200_ES.csv", index=False)